In [14]:
import scipy.io as sio
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import pickle
import os
import time
from sklearn.manifold import SpectralEmbedding
from sklearn.gaussian_process import GaussianProcessRegressor

In [4]:
datapath_base = "/data/yutaro/IROS/"
datapath_big = "/data/yutaro/IROS/sim_data_full_v11_d4_m1.mat"
datapath_small = "/data/yutaro/IROS/sim_data_partial_v111_d4_m1.mat"

big = sio.loadmat(datapath_big)
print(big['D'].shape)
print(pd.DataFrame(big['D']).head())

small = sio.loadmat(datapath_small)
print(small['D'].shape)
print(pd.DataFrame(small['D']).head())

(1616064, 10)
          0           1          2          3    4    5         6           7  \
0  0.426909  118.443377  16.000000  16.000000  0.0  0.0  0.418931  118.442513   
1  0.418931  118.442513  16.030001  16.030001  1.0  1.0  0.404663  118.443087   
2  0.404663  118.443087  16.030001  16.030001  1.0  1.0  0.396586  118.443348   
3  0.396586  118.443348  16.060001  16.060001  1.0  1.0  0.391399  118.442141   
4  0.391399  118.442141  16.060001  16.060001  1.0  1.0  0.384889  118.439220   

           8          9  
0  16.030001  16.030001  
1  16.030001  16.030001  
2  16.060001  16.060001  
3  16.060001  16.060001  
4  16.090002  16.090002  
(83888, 10)
          0         1         2          3    4    5         6         7  \
0  0.000215  0.118160 -0.005951  16.030001  1.0 -1.0  0.000233  0.118159   
1  0.000233  0.118159 -0.005951  16.030001  1.0 -1.0  0.000257  0.118159   
2  0.000257  0.118159 -0.005951  16.060001  1.0 -1.0  0.000295  0.118159   
3  0.000295  0.118159 -0.00

In [9]:
% time
nearest_neighbor(small['D'], small['D'])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


(array([0., 0., 0., ..., 0., 0., 0.]),
 array([    0,     1,     2, ..., 83885, 83886, 83887]))

In [8]:
small['D'].shape

(83888, 10)

In [27]:
np.mean(small['D'], axis=0).shape

(10,)

In [12]:
def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids = mean centered
    centroid_A = np.mean(A, axis=0) # centroid_A.shape = (m,)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H) 
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t

In [2]:
def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()

In [10]:
def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source 
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

In [19]:
reduced_n = small['D'].shape[0]

In [23]:
print(big['D'][:reduced_n, :].shape, small['D'].shape)

(83888, 10) (83888, 10)


In [24]:
# Run ICP
start = time.time()
T, distances, iterations = icp(big['D'][:reduced_n, :], small['D'], tolerance=0.000001)
print(time.time() - start)

107.83271026611328


In [18]:
iterations

0

In [26]:
T.shape

(11, 11)

In [29]:
big['D'][:reduced_n, :].shape[0]

83888

In [30]:
m = big['D'][:reduced_n, :].shape[1]
C = np.ones([big['D'][:reduced_n, :].shape[0],m+1])
C[:,0:m] = np.copy(big['D'][:reduced_n, :])
C = np.dot(T, C.T).T


# big['D'][reduced_n:(reduced_n+1), :]

In [34]:
pd.DataFrame(big['D'][:reduced_n, :]).head()

,0,1,2,3,4,5,6,7,8,9
0,0.426909,118.443377,16.000000,16.000000,0.0,0.0,0.418931,118.442513,16.030001,16.030001
1,0.418931,118.442513,16.030001,16.030001,1.0,1.0,0.404663,118.443087,16.030001,16.030001
2,0.404663,118.443087,16.030001,16.030001,1.0,1.0,0.396586,118.443348,16.060001,16.060001
3,0.396586,118.443348,16.060001,16.060001,1.0,1.0,0.391399,118.442141,16.060001,16.060001
4,0.391399,118.442141,16.060001,16.060001,1.0,1.0,0.384889,118.439220,16.090002,16.090002


In [33]:
pd.DataFrame(C).head() 

,0,1,2,3,4,5,6,7,8,9,10
0,0.760110,0.260586,0.300565,54.744640,-14.518222,6.235257,0.717201,0.248920,0.332193,55.724400,1.0
1,-0.255656,0.135269,0.271478,54.644751,-14.559483,6.299387,-0.237627,0.143635,0.310243,55.845564,1.0
2,-0.236395,0.144111,0.272911,54.654161,-14.534284,6.316442,-0.258423,0.139153,0.306494,55.857271,1.0
3,-0.256323,0.138483,0.267491,54.659783,-14.508554,6.332363,-0.239094,0.146890,0.310122,55.863741,1.0
4,-0.237107,0.144210,0.271107,54.665055,-14.481660,6.346394,-0.259760,0.139293,0.308153,55.868992,1.0
